In [1]:

from connectors.postgres_connector import PostgresConnector
from feeds.postgres_feed import PostgresFeed

import dedupe

pg_conn = PostgresConnector(db='defaultdb', username='doadmin', password='AVNS_2Lh_hY8r7RVKSfLwbJM', host='silobuster-dev-db-do-user-12298230-0.b.db.ondigitalocean.com', port=25060)
# select_qry = "SELECT id, name, address FROM what_location WHERE address is not null and not address = '' and name is not null and not name = ''"
# select_qry2 = "SELECT id, name, address FROM what_location"

# select_qry = "SELECT id, name, address_1, address_2, city, state_province, postal_code, url, description, duplicate_id, duplicate_type FROM organizations_normalized WHERE name IS NOT NULL AND NOT name = '' AND address_1 IS NOT NULL AND NOT address_1 = '' AND city IS NOT NULL AND NOT city = ''"

select_qry = '''SELECT
  o.id as organization_id,
  o.name as o_name,
  o.description as o_description,
  o.url as o_url,
  l.id as location_id,
  l.name as l_name,
  l.description as l_description,
  l.latitude,
  l.longitude,
  a.address_1,
  a.city,
  a.state_province,
  a.postal_code
FROM organization o
join location l on l.organization_id = o.id
join address a on a.location_id = l.id
  where type = 'physical';
'''
raw_def = [
    {"field": "name", "type": "String", "hass missing": True},
    {"field": "address_1", "type": "String", "has missing": True},
    {"field": "address_2", "type": "Exists", "has missing": True},
    {"field": "city", "type": "String", "has missing": True},
    {"field": "state_province", "type": "String", "has missing": True},
    {"field": "postal_code", "type": "Exists", "has missing": True},
    {"field": "url", "type": "Exists", "has missing": True}
]


pg_feed = PostgresFeed.from_manual(connector=pg_conn, query=select_qry, column_definition=raw_def, primary_key='id')

# select_qry = "SELECT name, address FROM what_location WHERE address is not null and not address = '' and name is not null and not name = ''"
deduper = dedupe.Dedupe(pg_feed.variable_definition)



In [2]:
#deduper.prepare_training(pg_feed.formatted_data, sample_size=100000, blocked_proportion=1)
deduper.prepare_training(pg_feed.formatted_data)

ValueError: Records do not line up with data model. The field 'name' is in data_model but not in a record

In [ ]:
#distinct_pairs = []
#while True:
#    try:
#        pair = deduper.uncertain_pairs()
#        distinct_pairs.extend(pair)
#    except IndexError as e:
#        break
#    except Exception as e:
#        raise

#deduper.mark_pairs({
#    "match": [],
#    "distinct": distinct_pairs
#})
#print (distinct_pairs)

In [ ]:
print("Starting active labeling...")

In [ ]:
dedupe.convenience.console_label(deduper)

print ('Dumping training results...')


In [ ]:
with open("training_results.txt", "w") as f:
    deduper.write_training(f)

In [ ]:
#with open("training_settings.txt", "wb") as f:
#    deduper.write_settings(f)

In [ ]:
#for field in deduper.fingerprinter.index_fields:
print ('training')
deduper.train() 
print('trained')